# Introducción al Razonamiento en LLMs


## Instalación de paquetes
Si estás corriendo este notebook en Google Colab, corre la siguiente celda para instalar los paquetes necesarios.

In [1]:
# %pip install langchain langchain_community langchain_openai

In [2]:
# Corre esta celda solo si tienes un archivo .env configurado
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
from langchain_openai import ChatOpenAI 
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatOpenAI(
    model=os.getenv("MODEL"),
    openai_api_key=os.getenv("LIA_API_KEY"),
    openai_api_base=os.getenv("LIA_API_BASE"),
    max_tokens=2000,
    temperature=0.6,
)

## Un primer ejemplo

In [4]:
# Palindromos

messages = [
    SystemMessage(
        content="Suma los palíndromos en la siguiente secuencia. No escribas el razonamiento, solo el resultado."
    ),
    HumanMessage(
        content="13, 1331, 121, 73, 99, 56, 232, 7"
    ),
]

llm.invoke(messages).content

'El resultado es 1261.'

### One-shot learning (aprendizaje de una sola muestra)

In [5]:
messages = [
    SystemMessage(
        content="Suma los palíndromos en la siguiente secuencia. Ejemplo: 33 es un palíndromo. No escribas el razonamiento, solo el resultado."
    ),
    HumanMessage(
        content="13, 1331, 121, 73, 99, 56, 232, 7"
    ),
]

llm.invoke(messages).content

'El resultado es 1261.'

Se dice two-shot learning cuando se necesita de dos ejemplos para aprender una tarea. En el caso anterior, se necesitó de un ejemplo para entender qué es un palíndromo y otro para entender cómo sumarlos. Three-shot learning sería cuando se necesita de tres ejemplos, y así sucesivamente.

### Few-shot training (entrenamiento con pocas muestras)

In [6]:
# Hagamos una prueba sin few-shot training

messages = [
    SystemMessage(
        content="Eres un experto en clasificar números como Abra, Kadabra o Abra Kadabra. El usuario te dará una serie de números y tú debes clasificarlos."
    ),
    HumanMessage(
        content="3, 5, 8, 2, 7, 12, 13, 35"
    ),
]

llm.invoke(messages).content

'Para clasificar los números que has proporcionado, aplicaremos las siguientes reglas:\n\n- **Abra**: Números impares.\n- **Kadabra**: Números pares.\n- **Abra Kadabra**: Números que son tanto impares como pares, es decir, no se aplica.\n\nAhora clasifiquemos los números:\n\n- 3: Abra (impar)\n- 5: Abra (impar)\n- 8: Kadabra (par)\n- 2: Kadabra (par)\n- 7: Abra (impar)\n- 12: Kadabra (par)\n- 13: Abra (impar)\n- 35: Abra (impar)\n\nClasificación final:\n- Abra: 3, 5, 7, 13, 35\n- Kadabra: 8, 2, 12\n- Abra Kadabra: Ninguno\n\nSi tienes más números para clasificar, ¡estaré encantado de ayudarte!'

In [7]:
# Ahora con few-shot training

messages = [
    SystemMessage(
        content="Eres un experto en clasificar números como Abra, Kadabra o Abra Kadabra. El usuario te dará una serie de números y tú debes clasificarlos según estas reglas: si es divisible por 5 es 'Abra', si es divisible por 7 es 'Kadabra', y si es divisible por ambos es 'Abra Kadabra'. Ejemplos: 10 es divisible por 5, es 'Abra'. 14 es divisible por 7 es 'Kadabra'. 70 es divisible por 5 y 7 es 'Abra Kadabra'.  Si no es divisible por ninguno, entonces no se clasifica."
    ),
    HumanMessage(
        content="3, 5, 8, 2, 7, 12, 13, 35"
    ),
] 

llm.invoke(messages).content

'Aquí está la clasificación de los números que proporcionaste:\n\n- 3: No se clasifica\n- 5: Abra\n- 8: No se clasifica\n- 2: No se clasifica\n- 7: Kadabra\n- 12: No se clasifica\n- 13: No se clasifica\n- 35: Abra\n\nResumen:\n- 5: Abra\n- 7: Kadabra\n- 35: Abra'

### Few-shot con Langchain

In [8]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# Ejemplos de clasificación
examples = [
  {
      "number": 6,
      "reasoning": "no divisible por 5 ni por 7",
      "result": "Ninguno"
  },
  {
      "number": 15,
      "reasoning": "divisible por 5 pero no por 7",
      "result": "Abra"
  },
  {
      "number": 12,
      "reasoning": "no divisible por 5 ni por 7",
      "result": "Ninguno"
  },
  {
      "number": 21,
      "reasoning": "divisible por 7 pero no por 5",
      "result": "Kadabra"
  },
  {
      "number": 70,
      "reasoning": "divisible por 5 y por 7",
      "result": "Abra Kadabra"
  } 
]

# Plantilla para los ejemplos
example_prompt = PromptTemplate(input_variables=["number", "reasoning", "result"], template="{number} \\ {reasoning} \\ {result}")

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Clasifica los siguientes números como Abra, Kadabra o Abra Kadabra: {comma_delimited_input_numbers}",
    input_variables=["comma_delimited_input_numbers"]
)

# Números para clasificar
prompt_input = few_shot_prompt.format(comma_delimited_input_numbers="3, 4, 5, 7, 8, 10, 11, 13, 35.")

# Invocar el modelo
response = llm.invoke(prompt_input)
print(response.content)


Vamos a clasificar los números según las reglas que proporcionaste:

1. **Números no divisibles por 5 ni por 7**: Ninguno
2. **Números divisibles por 5 pero no por 7**: Abra
3. **Números divisibles por 7 pero no por 5**: Kadabra
4. **Números divisibles por 5 y por 7**: Abra Kadabra

Ahora, analicemos cada número de la lista: 3, 4, 5, 7, 8, 10, 11, 13, 35.

- **3**: No es divisible por 5 ni por 7 → Ninguno
- **4**: No es divisible por 5 ni por 7 → Ninguno
- **5**: Divisible por 5 pero no por 7 → Abra
- **7**: Divisible por 7 pero no por 5 → Kadabra
- **8**: No es divisible por 5 ni por 7 → Ninguno
- **10**: Divisible por 5 pero no por 7 → Abra
- **11**: No es divisible por 5 ni por 7 → Ninguno
- **13**: No es divisible por 5 ni por 7 → Ninguno
- **35**: Divisible por 5 y por 7 → Abra Kadabra

Clasificación final:

- 3: Ninguno
- 4: Ninguno
- 5: Abra
- 7: Kadabra
- 8: Ninguno
- 10: Abra
- 11: Ninguno
- 13: Ninguno
- 35: Abra Kadabra


### Cadena de pensamiento - Chain of Thought (CoT)

Ahora retomemos el ejemplo de los palíndromos.

Vamos a crear un prompt utilizando la técnica del **Chain of Thought**, como fue introducida por Wei et al (https://arxiv.org/abs/2201.11903). Este prompt incluirá:

- Varias secuencias de ejemplo para el entrenamiento few-shot
- Una explicación de los pasos lógicos necesarios para determinar si una secuencia es "Extraña" (proporcionando los pasos)

In [9]:
# Identificación y suma de palíndromos con CoT (Chain of Thought)

messages = [
    SystemMessage(
        content="""Eres un experto en sumar palíndromos. Utilizarás el siguiente enfoque paso a paso para identificar y sumar los palíndromos de una secuencia. Ejemplos:

Ejemplo 1:
33 es un palíndromo
44 es un palíndromo

Pasos para resolver:
1) Identificar los palíndromos
2) Sumarlos

Ejemplo resuelto:

Secuencia: 1331, 121, 99, 232, 7
Paso 1: Identificar los palíndromos en la secuencia.
Palíndromos identificados: 1331, 121, 99, 232, 7

Paso 2: Sumar los palíndromos.
Cálculo: 1331 + 121 + 99 + 232 + 7 = 1790

Por lo tanto, la suma de los palíndromos es 1790.
"""
    ),
    HumanMessage(
        content="""Suma los palíndromos en la siguiente secuencia: 24, 1331, 121, 89, 55, 32, 767, 2."""
    ),
]

response = llm.invoke(messages).content
print(response)


Vamos a seguir los pasos para identificar y sumar los palíndromos en la secuencia dada.

Secuencia: 24, 1331, 121, 89, 55, 32, 767, 2

**Paso 1: Identificar los palíndromos en la secuencia.**
- 24: no es un palíndromo
- 1331: es un palíndromo
- 121: es un palíndromo
- 89: no es un palíndromo
- 55: es un palíndromo
- 32: no es un palíndromo
- 767: es un palíndromo
- 2: es un palíndromo

**Palíndromos identificados:** 1331, 121, 55, 767, 2

**Paso 2: Sumar los palíndromos.**
Cálculo: 1331 + 121 + 55 + 767 + 2 = 2276

Por lo tanto, la suma de los palíndromos es **2276**.


### Buenas prácticas para la ingeniería de prompts

Con lo aprendido anteriormente, podemos combinar todos los elementos del prompt de las secciones previas para obtener la siguiente estructura general:

**Persona**: Especifica el rol que deseas que el modelo de lenguaje (LLM) asuma. Ya sea un gerente de marketing experimentado, un autor galardonado de ciencia ficción o un editor de texto competente para una revista de informática, establece expectativas altas.

**Contexto**: Proporciona información detallada para ayudar al LLM a comprender el contexto de tu solicitud. La precisión aumenta la probabilidad de que el resultado se alinee con tus expectativas.

**Instrucción**: Define claramente la acción que deseas que el LLM realice con tu texto de entrada. Ya sea traducir, escribir, reescribir, mejorar, clasificar, ordenar o encontrar, sé explícito.

**Entrada**: Se refiere a detalles específicos del contexto, que pueden ser un párrafo, una consulta o una lista de puntos clave. Puedes etiquetarlo como "Texto", "Pregunta", etc.

**Pasos**: Describe los pasos de procesamiento que el LLM debe seguir para generar la salida.

**Tono**: Especifica el tono deseado en la respuesta del LLM—formal, informal, ingenioso, entusiasta, sobrio, amigable, etc. Las combinaciones son posibles.

**Formato de salida**: Opcionalmente, indica el formato de salida deseado. Si no se especifica, el LLM asumirá una respuesta en texto. Sin embargo, puedes solicitar formatos como una tabla, archivo CSV (con nombres de columnas), JSON (con nombres de atributos) u otros formatos estructurados.

**Ejemplos**: En casos donde el LLM pueda carecer de suficiente entrenamiento para ciertas tareas, especialmente en clasificación de texto y razonamiento, proporciona entrenamiento en línea a través de ejemplos. Incluso unos pocos ejemplos mejoran significativamente la calidad de la respuesta.